# SUTD 2021 50.007 Machine Learning HMM Project Part 1

In [ ]:
# Setup and install dependencies
!pip3 install numpy

# Import libraries
import os
import numpy as np

# Set OS-independent paths, relative to current directory
es_dev_in_path = os.path.join("data", "ES", "dev.in")
es_dev_out_path = os.path.join("data", "ES", "dev.out")
ru_dev_in_path = os.path.join("data", "RU", "dev.in")
ru_dev_out_path = os.path.join("data", "RU", "dev.out")

In [ ]:
# Read dev.in data
def read_dev_in_data(filepath):
    results = []
    with open(filepath, "r", encoding="utf-8") as file:
        lines = file.readlines()
        for line in lines:
            if line.strip():
                token = line.strip()
                results.append(token)
            else:
                continue
    return results

# Read dev.out data
def read_dev_out_data(filepath):
    results = []
    with open(filepath, "r", encoding="utf-8") as file:
        lines = file.readlines()
        for line in lines:
            if len(line.strip().rsplit(" ", 1)) == 2:
                token, label = line.strip().rsplit(" ", 1)
                results.append((token, label))
            else:
                continue
    return results